# Xây dựng mô hình hồi quy: chuẩn bị và trực quan hóa dữ liệu

## **Hồi quy tuyến tính cho bí ngô - Bài học 2**
#### Giới thiệu

Bây giờ bạn đã có các công cụ cần thiết để bắt đầu xây dựng mô hình học máy với Tidymodels và Tidyverse, bạn đã sẵn sàng để bắt đầu đặt câu hỏi về dữ liệu của mình. Khi làm việc với dữ liệu và áp dụng các giải pháp ML, điều rất quan trọng là phải hiểu cách đặt câu hỏi đúng để khai thác tối đa tiềm năng của tập dữ liệu.

Trong bài học này, bạn sẽ học:

-   Cách chuẩn bị dữ liệu của bạn để xây dựng mô hình.

-   Cách sử dụng `ggplot2` để trực quan hóa dữ liệu.

Câu hỏi bạn cần trả lời sẽ quyết định loại thuật toán ML mà bạn sẽ sử dụng. Và chất lượng của câu trả lời bạn nhận được sẽ phụ thuộc rất nhiều vào bản chất của dữ liệu.

Hãy cùng xem điều này qua một bài tập thực hành.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Tác phẩm nghệ thuật của @allison_horst</figcaption>


<!--![Tác phẩm nghệ thuật của \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Tác phẩm nghệ thuật của \@allison_horst-->


## 1. Nhập dữ liệu về bí ngô và triệu hồi Tidyverse

Chúng ta sẽ cần các gói sau để phân tích và xử lý bài học này:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) là một [bộ sưu tập các gói R](https://www.tidyverse.org/packages) được thiết kế để làm cho khoa học dữ liệu trở nên nhanh hơn, dễ dàng hơn và thú vị hơn!

Bạn có thể cài đặt chúng bằng cách:

`install.packages(c("tidyverse"))`

Đoạn mã dưới đây kiểm tra xem bạn đã có các gói cần thiết để hoàn thành module này chưa và sẽ cài đặt chúng cho bạn nếu thiếu.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Bây giờ, hãy khởi động một số gói và tải [dữ liệu](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) được cung cấp cho bài học này!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Một `glimpse()` nhanh chóng cho thấy rằng có các giá trị trống và sự kết hợp giữa chuỗi ký tự (`chr`) và dữ liệu số (`dbl`). Cột `Date` thuộc kiểu ký tự và còn có một cột kỳ lạ tên là `Package`, nơi dữ liệu là sự pha trộn giữa `sacks`, `bins` và các giá trị khác. Thực tế, dữ liệu này khá lộn xộn 😤.

Thực tế, không thường xuyên bạn nhận được một tập dữ liệu hoàn toàn sẵn sàng để sử dụng nhằm tạo ra một mô hình ML ngay lập tức. Nhưng đừng lo, trong bài học này, bạn sẽ học cách chuẩn bị một tập dữ liệu thô bằng cách sử dụng các thư viện R tiêu chuẩn 🧑‍🔧. Bạn cũng sẽ học các kỹ thuật khác nhau để trực quan hóa dữ liệu. 📈📊
<br>

> Một lời nhắc lại: Toán tử pipe (`%>%`) thực hiện các thao tác theo trình tự logic bằng cách chuyển một đối tượng vào một hàm hoặc biểu thức gọi. Bạn có thể nghĩ toán tử pipe như đang nói "và sau đó" trong mã của bạn.


## 2. Kiểm tra dữ liệu bị thiếu

Một trong những vấn đề phổ biến nhất mà các nhà khoa học dữ liệu phải đối mặt là dữ liệu không đầy đủ hoặc bị thiếu. R biểu thị các giá trị bị thiếu hoặc không xác định bằng một giá trị đặc biệt: `NA` (Not Available).

Vậy làm thế nào để chúng ta biết rằng khung dữ liệu chứa các giá trị bị thiếu?
<br>
-   Một cách đơn giản là sử dụng hàm cơ bản của R `anyNA`, hàm này trả về các đối tượng logic `TRUE` hoặc `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Tuyệt vời, có vẻ như đang thiếu một số dữ liệu! Đây là một điểm tốt để bắt đầu.

-   Một cách khác là sử dụng hàm `is.na()` để xác định các phần tử bị thiếu trong từng cột với giá trị logic `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Được rồi, đã hoàn thành công việc nhưng với một khung dữ liệu lớn như thế này, việc xem xét từng hàng và cột riêng lẻ sẽ không hiệu quả và gần như không thể😴.

-   Một cách trực quan hơn là tính tổng số giá trị bị thiếu cho mỗi cột:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Tốt hơn nhiều! Có dữ liệu bị thiếu, nhưng có lẽ điều đó sẽ không ảnh hưởng đến nhiệm vụ hiện tại. Hãy xem phân tích tiếp theo sẽ mang lại điều gì.

> Bên cạnh các bộ gói và hàm tuyệt vời, R còn có tài liệu hướng dẫn rất tốt. Ví dụ, sử dụng `help(colSums)` hoặc `?colSums` để tìm hiểu thêm về hàm này.


## 3. Dplyr: Ngữ pháp của việc xử lý dữ liệu

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Tác phẩm nghệ thuật của @allison_horst</figcaption>


<!--![Tác phẩm nghệ thuật của \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Tác phẩm nghệ thuật của \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), một gói trong Tidyverse, là một ngữ pháp xử lý dữ liệu cung cấp một tập hợp các động từ nhất quán giúp bạn giải quyết các thách thức xử lý dữ liệu phổ biến nhất. Trong phần này, chúng ta sẽ khám phá một số động từ của dplyr!  
<br>  


#### dplyr::select()

`select()` là một hàm trong gói `dplyr` giúp bạn chọn các cột để giữ lại hoặc loại bỏ.

Để làm cho khung dữ liệu của bạn dễ làm việc hơn, hãy loại bỏ một số cột bằng cách sử dụng `select()`, chỉ giữ lại các cột bạn cần.

Ví dụ, trong bài tập này, phân tích của chúng ta sẽ liên quan đến các cột `Package`, `Low Price`, `High Price` và `Date`. Hãy chọn các cột này.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` là một hàm trong gói `dplyr`, giúp bạn tạo hoặc chỉnh sửa các cột, đồng thời giữ nguyên các cột hiện có.

Cấu trúc chung của `mutate` là:

`data %>%   mutate(new_column_name = what_it_contains)`

Hãy thử sử dụng `mutate` với cột `Date` bằng cách thực hiện các thao tác sau:

1. Chuyển đổi các ngày (hiện tại thuộc kiểu ký tự) sang định dạng tháng (đây là ngày tháng kiểu Mỹ, nên định dạng là `MM/DD/YYYY`).

2. Trích xuất tháng từ các ngày và lưu vào một cột mới.

Trong R, gói [lubridate](https://lubridate.tidyverse.org/) giúp làm việc với dữ liệu ngày giờ dễ dàng hơn. Vì vậy, hãy sử dụng `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` để đạt được các mục tiêu trên. Chúng ta có thể loại bỏ cột `Date` vì sẽ không cần sử dụng nó nữa trong các thao tác tiếp theo.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Woohoo! 🤩

Tiếp theo, hãy tạo một cột mới `Price`, đại diện cho giá trung bình của một quả bí ngô. Bây giờ, hãy lấy trung bình của các cột `Low Price` và `High Price` để điền vào cột Price mới.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

"Nhưng khoan đã!", bạn sẽ nói sau khi lướt qua toàn bộ tập dữ liệu với `View(pumpkins)`, "Có điều gì đó kỳ lạ ở đây!"🤔

Nếu bạn nhìn vào cột `Package`, bí ngô được bán theo nhiều cách khác nhau. Một số được bán theo đơn vị `1 1/9 bushel`, một số theo đơn vị `1/2 bushel`, một số theo từng quả bí ngô, một số theo cân nặng, và một số được đóng trong các hộp lớn với kích thước khác nhau.

Hãy kiểm tra điều này:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Tuyệt vời!👏

Bí ngô dường như rất khó để cân một cách nhất quán, vì vậy hãy lọc chúng bằng cách chỉ chọn những quả bí ngô có chuỗi *bushel* trong cột `Package` và đặt chúng vào một khung dữ liệu mới `new_pumpkins`.


#### dplyr::filter() và stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): tạo một tập con của dữ liệu chỉ chứa **các hàng** thỏa mãn điều kiện của bạn, trong trường hợp này là các hàng có từ *bushel* trong cột `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): phát hiện sự có mặt hoặc vắng mặt của một mẫu trong chuỗi.

Gói [`stringr`](https://github.com/tidyverse/stringr) cung cấp các hàm đơn giản cho các thao tác chuỗi thông dụng.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Bạn có thể thấy rằng chúng tôi đã thu hẹp xuống còn khoảng 415 dòng dữ liệu chứa bí ngô theo giạ.🤩
<br>


#### dplyr::case_when()

**Nhưng khoan đã! Còn một việc nữa cần làm**

Bạn có nhận thấy rằng số lượng giạ thay đổi theo từng hàng không? Bạn cần chuẩn hóa giá để hiển thị giá theo mỗi giạ, không phải theo 1 1/9 hay 1/2 giạ. Đã đến lúc thực hiện một số phép toán để chuẩn hóa.

Chúng ta sẽ sử dụng hàm [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) để *biến đổi* cột Price dựa trên một số điều kiện. `case_when` cho phép bạn vector hóa nhiều câu lệnh `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Bây giờ, chúng ta có thể phân tích giá theo đơn vị dựa trên đo lường bằng giạ. Tuy nhiên, tất cả việc nghiên cứu về giạ bí ngô này cho thấy rằng việc `hiểu rõ bản chất của dữ liệu` của bạn là điều `rất quan trọng`!

> ✅ Theo [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), trọng lượng của một giạ phụ thuộc vào loại nông sản, vì đây là một đơn vị đo thể tích. "Ví dụ, một giạ cà chua được cho là nặng 56 pound... Lá và rau xanh chiếm nhiều không gian hơn nhưng lại nhẹ hơn, vì vậy một giạ rau chân vịt chỉ nặng 20 pound." Thật là phức tạp! Chúng ta không cần phải bận tâm đến việc chuyển đổi từ giạ sang pound, thay vào đó hãy định giá theo giạ. Tuy nhiên, tất cả việc nghiên cứu về giạ bí ngô này cho thấy rằng việc hiểu rõ bản chất của dữ liệu của bạn là điều rất quan trọng!
>
> ✅ Bạn có để ý rằng bí ngô được bán theo nửa giạ có giá rất đắt không? Bạn có thể tìm ra lý do tại sao không? Gợi ý: những quả bí ngô nhỏ đắt hơn rất nhiều so với những quả lớn, có lẽ vì có nhiều quả nhỏ hơn trong một giạ, do không gian bị chiếm bởi một quả bí ngô lớn rỗng ruột.


Bây giờ cuối cùng, chỉ để thêm phần thú vị 💁‍♀️, hãy di chuyển cột Month lên vị trí đầu tiên, tức là `trước` cột `Package`.

`dplyr::relocate()` được sử dụng để thay đổi vị trí các cột.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Làm tốt lắm!👌 Giờ bạn đã có một tập dữ liệu sạch sẽ, gọn gàng để xây dựng mô hình hồi quy mới của mình!  
<br>


## 4. Trực quan hóa dữ liệu với ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Đồ họa thông tin bởi Dasani Madipalli</figcaption>


<!--![Đồ họa thông tin bởi Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Có một câu nói *thông thái* như sau:

> "Biểu đồ đơn giản đã mang lại nhiều thông tin hơn cho nhà phân tích dữ liệu so với bất kỳ công cụ nào khác." --- John Tukey

Một phần vai trò của nhà khoa học dữ liệu là thể hiện chất lượng và bản chất của dữ liệu mà họ đang làm việc. Để làm điều này, họ thường tạo ra các hình ảnh trực quan thú vị, hoặc các biểu đồ, đồ thị, và sơ đồ, thể hiện các khía cạnh khác nhau của dữ liệu. Bằng cách này, họ có thể trực quan hóa các mối quan hệ và khoảng trống mà nếu không sẽ khó phát hiện.

Các hình ảnh trực quan cũng có thể giúp xác định kỹ thuật học máy phù hợp nhất với dữ liệu. Ví dụ, một biểu đồ phân tán có xu hướng theo một đường thẳng cho thấy dữ liệu là ứng viên tốt cho bài toán hồi quy tuyến tính.

R cung cấp một số hệ thống để tạo biểu đồ, nhưng [`ggplot2`](https://ggplot2.tidyverse.org/index.html) là một trong những hệ thống thanh lịch và linh hoạt nhất. `ggplot2` cho phép bạn tạo biểu đồ bằng cách **kết hợp các thành phần độc lập**.

Hãy bắt đầu với một biểu đồ phân tán đơn giản cho các cột Price và Month.

Trong trường hợp này, chúng ta sẽ bắt đầu với [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), cung cấp một tập dữ liệu và ánh xạ thẩm mỹ (với [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) sau đó thêm các lớp (như [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) cho biểu đồ phân tán.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Đây có phải là một biểu đồ hữu ích không 🤷? Có điều gì khiến bạn ngạc nhiên về nó không?

Nó không thực sự hữu ích vì tất cả những gì nó làm chỉ là hiển thị dữ liệu của bạn dưới dạng một loạt các điểm trong một tháng nhất định.  
<br>


### **Làm thế nào để chúng ta làm cho nó hữu ích?**

Để biểu đồ hiển thị dữ liệu hữu ích, bạn thường cần nhóm dữ liệu theo một cách nào đó. Ví dụ, trong trường hợp của chúng ta, tìm giá trung bình của bí ngô theo từng tháng sẽ cung cấp thêm thông tin chi tiết về các mẫu ẩn trong dữ liệu. Điều này dẫn chúng ta đến một công cụ khác của **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Việc tính toán tổng hợp theo nhóm trong R có thể được thực hiện dễ dàng bằng cách sử dụng

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` thay đổi đơn vị phân tích từ toàn bộ tập dữ liệu sang các nhóm riêng lẻ, chẳng hạn như theo từng tháng.

-   `dplyr::summarize()` tạo một khung dữ liệu mới với một cột cho mỗi biến nhóm và một cột cho mỗi thống kê tóm tắt mà bạn đã chỉ định.

Ví dụ, chúng ta có thể sử dụng `dplyr::group_by() %>% summarize()` để nhóm các bí ngô thành các nhóm dựa trên cột **Month** và sau đó tìm **giá trung bình** cho từng tháng.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Ngắn gọn!✨

Các đặc điểm phân loại như tháng được biểu diễn tốt hơn bằng biểu đồ cột 📊. Các lớp chịu trách nhiệm cho biểu đồ cột là `geom_bar()` và `geom_col()`. Tham khảo `?geom_bar` để tìm hiểu thêm.

Hãy cùng tạo một cái nhé!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 Đây là một biểu đồ trực quan hóa dữ liệu hữu ích hơn! Dường như nó chỉ ra rằng giá cao nhất của bí ngô xảy ra vào tháng 9 và tháng 10. Điều này có đúng với mong đợi của bạn không? Tại sao hoặc tại sao không?

Chúc mừng bạn đã hoàn thành bài học thứ hai 👏! Bạn đã chuẩn bị dữ liệu của mình để xây dựng mô hình, sau đó khám phá thêm nhiều thông tin chi tiết bằng cách sử dụng biểu đồ trực quan hóa!



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
